# imports

In [5]:
import sys
import os
import numpy as np
import pandas as pd
# mcfly
from mcfly import modelgen, find_architecture, storage
from keras.models import load_model
np.random.seed(2)

In [6]:
sys.path.insert(0, os.path.abspath('../..'))
from utils import tutorial_vu

# Load data

In [8]:
X_train, y_train, X_val, y_val, X_test, y_test = tutorial_vu.load_multi_time_fall_dataset(base_path='/data/Data espen/')

X_train = X_train[:, 1000:1500, :]
X_val = X_val[:, 1000:1500, :]
X_test = X_test[:, 1000:1500, :]

Loading training data.
Processing file Acc_Vel_gait_30sec_170.mat
Processing file Acc_Vel_gait_30sec_106.mat
Processing file Acc_Vel_gait_30sec_17.mat
Processing file Acc_Vel_gait_30sec_54.mat
Processing file Acc_Vel_gait_30sec_141.mat
Processing file Acc_Vel_gait_30sec_68.mat
Processing file Acc_Vel_gait_30sec_40.mat
Processing file Acc_Vel_gait_30sec_121.mat
Processing file Acc_Vel_gait_30sec_122.mat
Processing file Acc_Vel_gait_30sec_160.mat
Processing file Acc_Vel_gait_30sec_93.mat
Processing file Acc_Vel_gait_30sec_143.mat
Processing file Acc_Vel_gait_30sec_44.mat
Processing file Acc_Vel_gait_30sec_66.mat
Processing file Acc_Vel_gait_30sec_139.mat
Processing file Acc_Vel_gait_30sec_188.mat
Processing file Acc_Vel_gait_30sec_7.mat
Processing file Acc_Vel_gait_30sec_62.mat
Processing file Acc_Vel_gait_30sec_148.mat
Processing file Acc_Vel_gait_30sec_95.mat
Processing file Acc_Vel_gait_30sec_43.mat
Processing file Acc_Vel_gait_30sec_179.mat
Processing file Acc_Vel_gait_30sec_124.mat


In [9]:
from keras.utils.np_utils import to_categorical
y_train_binary = to_categorical(np.array(y_train>0, dtype='int'))
y_val_binary = to_categorical(np.array(y_val>0, dtype='int'))
y_test_binary = to_categorical(np.array(y_test>0, dtype='int'))

# Inspect data

In [10]:
print('train set size:', X_train.shape[0])
print('validation set size:', X_val.shape[0])
print('test set size:', X_test.shape[0])

train set size: 14581
validation set size: 1203
test set size: 2652


In [11]:
print('x shape:', X_train.shape)
print('y shape:', y_train_binary.shape)

x shape: (14581, 500, 6)
y shape: (14581, 2)


In [12]:
# Look at proportion fallers/ non fallers
y_val_binary.mean(axis=0)

array([ 0.50124688,  0.49875312])

# Generate models

In [13]:
num_classes = y_train_binary.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 5, model_type='CNN' )

Let's inspect the models:

In [14]:
models

[(<keras.models.Sequential at 0x7f27395447f0>,
  {'fc_hidden_nodes': 231,
   'filters': array([75, 36, 56, 78]),
   'learning_rate': 0.00042981092192911755,
   'regularization_rate': 0.00019438638666477383},
  'CNN'),
 (<keras.models.Sequential at 0x7f26ef641240>,
  {'fc_hidden_nodes': 111,
   'filters': array([45]),
   'learning_rate': 0.00030629849628779405,
   'regularization_rate': 0.023826992156800057},
  'CNN'),
 (<keras.models.Sequential at 0x7f26ef3dc208>,
  {'fc_hidden_nodes': 604,
   'filters': array([14, 70, 51, 81, 22, 37, 89, 32, 24]),
   'learning_rate': 0.019304098294381178,
   'regularization_rate': 0.026884467039248378},
  'CNN'),
 (<keras.models.Sequential at 0x7f26eea92e10>,
  {'fc_hidden_nodes': 704,
   'filters': array([49, 92, 45, 44, 42, 87]),
   'learning_rate': 0.004152660220244203,
   'regularization_rate': 0.00187529889376639},
  'CNN'),
 (<keras.models.Sequential at 0x7f26ee49f320>,
  {'fc_hidden_nodes': 1774,
   'filters': array([68, 73, 43]),
   'learning_

In [15]:
models[0][0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 500, 6)            24        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 75)           1425      
_________________________________________________________________
batch_normalization_2 (Batch (None, 500, 75)           300       
_________________________________________________________________
activation_1 (Activation)    (None, 500, 75)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 36)           8136      
_________________________________________________________________
batch_normalization_3 (Batch (None, 500, 36)           144       
_________________________________________________________________
activation_2 (Activation)    (None, 500, 36)           0         
__________

In [16]:

for i, item in enumerate(models):
    model, params, model_types = item
    print("-------------------------------------------------------------------------------------------------------")
    print("Model " + str(i))
    print(" ")
    print("Hyperparameters:")
    print(params)
    print(" ")
    print("Model description:")
    model.summary()
    print(" ")
    print("Model type:")
    print(model_types)
    print(" ")

-------------------------------------------------------------------------------------------------------
Model 0
 
Hyperparameters:
{'learning_rate': 0.00042981092192911755, 'regularization_rate': 0.00019438638666477383, 'filters': array([75, 36, 56, 78]), 'fc_hidden_nodes': 231}
 
Model description:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 500, 6)            24        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 75)           1425      
_________________________________________________________________
batch_normalization_2 (Batch (None, 500, 75)           300       
_________________________________________________________________
activation_1 (Activation)    (None, 500, 75)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500

# Compare the models

In [18]:
outputfile = 'modelcomparison.json'
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val[:100], y_val_binary[:100],
                                                                           models,nr_epochs=5,
                                                                           subset_size=14581,
                                                                           verbose=True,
                                                                           outputfile=outputfile,)
print('Details of the training process were stored in ',outputfile)

Training model 0 CNN
Train on 14581 samples, validate on 100 samples
Epoch 1/5
14581/14581 [==============================] - 46s - loss: 0.7812 - acc: 0.5034 - val_loss: 0.7528 - val_acc: 0.5800
Epoch 2/5
14581/14581 [==============================] - 44s - loss: 0.7487 - acc: 0.5283 - val_loss: 0.7419 - val_acc: 0.4800
Epoch 3/5
14581/14581 [==============================] - 44s - loss: 0.7362 - acc: 0.5381 - val_loss: 0.7288 - val_acc: 0.5200
Epoch 4/5
14581/14581 [==============================] - 44s - loss: 0.7333 - acc: 0.5444 - val_loss: 0.7375 - val_acc: 0.5000
Epoch 5/5
14581/14581 [==============================] - 44s - loss: 0.7319 - acc: 0.5645 - val_loss: 0.7549 - val_acc: 0.5300
Training model 1 CNN
Train on 14581 samples, validate on 100 samples
Epoch 1/5
14581/14581 [==============================] - 13s - loss: 1.4966 - acc: 0.4940 - val_loss: 0.9154 - val_acc: 0.4500
Epoch 2/5
14581/14581 [==============================] - 13s - loss: 0.7819 - acc: 0.4954 - val_loss

In [24]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
best_model.evaluate(X_val, y_val_binary, verbose=True)

2624/2652 [============================>.] - ETA: 0s

[0.75626566956665364, 0.4894419308881055]

In [25]:
history = best_model.fit(X_train, y_train_binary,
              epochs=20, validation_data=(X_val, y_val_binary))

Train on 14581 samples, validate on 1203 samples
Epoch 1/20
14581/14581 [==============================] - 32s - loss: 0.7091 - acc: 0.6141 - val_loss: 0.7891 - val_acc: 0.5071
Epoch 2/20
14581/14581 [==============================] - 32s - loss: 0.6829 - acc: 0.6569 - val_loss: 0.8239 - val_acc: 0.5461
Epoch 3/20
14581/14581 [==============================] - 32s - loss: 0.6477 - acc: 0.6977 - val_loss: 0.8745 - val_acc: 0.5245
Epoch 4/20
14581/14581 [==============================] - 32s - loss: 0.5867 - acc: 0.7480 - val_loss: 0.9856 - val_acc: 0.5245
Epoch 5/20
14581/14581 [==============================] - 32s - loss: 0.5104 - acc: 0.7959 - val_loss: 1.2599 - val_acc: 0.5137
Epoch 6/20
14581/14581 [==============================] - 32s - loss: 0.4299 - acc: 0.8491 - val_loss: 1.1285 - val_acc: 0.5270
Epoch 7/20
14581/14581 [==============================] - 32s - loss: 0.3600 - acc: 0.8833 - val_loss: 1.1550 - val_acc: 0.5162
Epoch 8/20
14581/14581 [==============================]

In [26]:
best_model.evaluate(X_test, y_test_binary, verbose=True)

2624/2652 [============================>.] - ETA: 0s

[1.8361225606449409, 0.50226244388842112]